In [1]:
from heeps.config import read_config, update_config
from heeps.util.img_processing import resize_cube
from astropy.io import fits
import numpy as np
import os

In [2]:
os.chdir(os.path.normpath(os.path.expandvars('$HOME/heeps_metis/input_files/wavefront/cbw')))
npupil_L = update_config(**dict(read_config(), band='L'))['npupil']
npupil_N = update_config(**dict(read_config(), band='N2'))['npupil']
npupil = {'L':npupil_L, 'N2':npupil_N}
tag = {'L':'LM', 'N2':'N'}
mask_L = fits.getdata('mask_Cfull_20220929_%s_%s.fits'%('L', npupil_L)) > 0.5
mask_N = fits.getdata('mask_Cfull_20220929_%s_%s.fits'%('N2', npupil_N)) > 0.5
mask_M1 = fits.getdata('mask_M1_511.fits') > 0.5
mask = {'L':mask_L, 'N2':mask_N}
date = '20221006'
filename = '%s/%s/%s_rep_%s_field_%s.fits.gz' # NCPA: DIFF = IMGP - SCAP ; Talbot: IMGA
nfields = 101
nreps = 10
med_rep = {'L':6, 'N2':5} # see chrom_beam_wander.ipynb notebook

In [3]:
# André's array is 512x512, centered at the upper-right pixel.
# We want odd size, so we don't include the first row and column.
# 512 --> 511

In [4]:
for band in ['L', 'N2']:
    for rep in range(1, nreps+1):#[1]
        if rep == med_rep[band]:
            starts = [0, 13, 25, 32, 38, 44]
            start_times = ['-2h00', '-1h30', '-1h00', '-0h45', '-0h30', '-0h15']
        else:
            starts = [38]
            start_times = ['-0h30']
        for start, start_time in zip(starts, start_times):
            
            # 1) Talbot effect
            write_name = '%s/talbot/%s_rep_%s_%s.fits'%(date, tag[band], rep, start_time)
            if os.path.isfile(write_name):
                print(write_name + ' already exists.')
            else:
                for i, field in enumerate(range(start, start+25)):
                    talbot = fits.getdata(filename%(date, tag[band], 'IMGA', rep, field))[1:,1:]
                    talbot = resize_cube(talbot, npupil[band])
                    #talbot[~mask[band]] = 0  # better not mask with zeros because HEEPS multiplies: proper.prop_multiply(wf, pad_img(amp_screen, ngrid))
                    stack = np.dstack([talbot.T]*480).T
                    if i == 0:
                        cube = stack
                    else:
                        cube = np.vstack((cube, stack))
                fits.writeto(write_name, cube)
                print(write_name + ' created.')

            # 2) NCPA
            write_name = '%s/ncpa/%s_rep_%s_%s.fits'%(date, tag[band], rep, start_time)
            if os.path.isfile(write_name):
                print(write_name + ' already exists.')
            else:
                for i, field in enumerate(range(start, start+25)):
                    IMGP = fits.getdata(filename%(date, tag[band], 'IMGP', rep, field))[1:,1:]
                    SCAP = fits.getdata(filename%(date, tag[band], 'SCAP', rep, field))[1:,1:]
                    ncpa = IMGP - SCAP
                    ncpa = resize_cube(ncpa, npupil[band])
                    ncpa -= np.mean(ncpa[mask[band]]) # remove piston
                    ncpa[~mask[band]] = 0             # mask with zeros
                    stack = np.dstack([ncpa.T]*480).T
                    if i == 0:
                        cube = stack
                    else:
                        cube = np.vstack((cube, stack))
                fits.writeto(write_name, cube)
                print(write_name + ' created.')

20221006/talbot/LM_rep_1_-0h30.fits already exists.
20221006/ncpa/LM_rep_1_-0h30.fits already exists.
20221006/talbot/LM_rep_2_-0h30.fits already exists.
20221006/ncpa/LM_rep_2_-0h30.fits already exists.
20221006/talbot/LM_rep_3_-0h30.fits already exists.
20221006/ncpa/LM_rep_3_-0h30.fits already exists.
20221006/talbot/LM_rep_4_-0h30.fits already exists.
20221006/ncpa/LM_rep_4_-0h30.fits already exists.
20221006/talbot/LM_rep_5_-0h30.fits already exists.
20221006/ncpa/LM_rep_5_-0h30.fits already exists.
20221006/talbot/LM_rep_6_-2h00.fits created.
20221006/ncpa/LM_rep_6_-2h00.fits created.
20221006/talbot/LM_rep_6_-1h30.fits created.
20221006/ncpa/LM_rep_6_-1h30.fits created.
20221006/talbot/LM_rep_6_-1h00.fits created.
20221006/ncpa/LM_rep_6_-1h00.fits created.
20221006/talbot/LM_rep_6_-0h45.fits created.
20221006/ncpa/LM_rep_6_-0h45.fits created.
20221006/talbot/LM_rep_6_-0h30.fits already exists.
20221006/ncpa/LM_rep_6_-0h30.fits already exists.
20221006/talbot/LM_rep_6_-0h15.fits 